In [ ]:
# Project 2

# Simulate arrivals


import numpy as np
arrivals=[]

def gen_arrivals():
    return arrivals.append(np.random.exponential())



0.79587450816311

In [ ]:
import numpy as np
import heapq
np.random.seed(0)
# ---
# Hospital Bed Allocation Simulation - Baseline Model
# ---

# --- 1. Define Model Parameters ---

# Using a dictionary for easy lookup by ward name
# ward_names maps an index (0-4) to a name ('A'-'E')
ward_names = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}
name_to_idx = {name: i for i, name in ward_names.items()}


# Bed capacities for each ward
capacities = {
    'A': 55, 'B': 40, 'C': 30, 'D': 20, 'E': 20
}

# Arrival rates (patients per day)
# This is lambda from the exercise
arrival_rates = {
    'A': 14.5, 'B': 11.0, 'C': 8.0, 'D': 6.5, 'E': 5.0
}

# Mean length-of-stay (in days)
# This is 1/mu from the exercise
mean_stay = {
    'A': 2.9, 'B': 4.0, 'C': 4.5, 'D': 1.4, 'E': 3.9
}

# Relocation probabilities when a ward is full
reloc_probs = {
    # If a Type 'A' patient arrives and Ward 'A' is full...
    'A': {'A': 0.0, 'B': 0.05, 'C': 0.10, 'D': 0.05, 'E': 0.80},
    'B': {'A': 0.2, 'B': 0.0, 'C': 0.50, 'D': 0.15, 'E': 0.15},
    'C': {'A': 0.3, 'B': 0.2, 'C': 0.0, 'D': 0.20, 'E': 0.30},
    'D': {'A': 0.35,'B': 0.3, 'C': 0.05, 'D': 0.0, 'E': 0.30},
    'E': {'A': 0.2, 'B': 0.1, 'C': 0.60, 'D': 0.1, 'E': 0.0}
}


# --- 2. Simulation State and Tracking ---

# Current number of occupied beds in each ward
occupied_beds = {name: 0 for name in capacities}

# List of future events, managed as a priority queue
# Format: (event_time, event_type, patient_type)
event_list = []

# Statistics trackers
total_arrivals = {name: 0 for name in capacities}
successful_admissions = {name: 0 for name in capacities}
relocated_from = {name: 0 for name in capacities} # Patients bumped from their primary ward
lost_patients = {name: 0 for name in capacities} # Patients lost after failing relocation


# --- 3. The Simulation Core ---

def schedule_arrival(patient_type, current_time):
    """Schedules the *next* arrival for a given patient type."""
    # Time to next arrival is from an exponential distribution
    time_delta = np.random.exponential(1.0 / arrival_rates[patient_type])
    arrival_time = current_time + time_delta
    # Add to event list (priority queue)
    heapq.heappush(event_list, (arrival_time, 'ARRIVAL', patient_type))
    
def schedule_departure(patient_type, current_time):
    """Schedules the departure for a patient who was just admitted."""
    stay_duration = np.random.exponential(mean_stay[patient_type])
    departure_time = current_time + stay_duration
    # We use patient_type here to know which ward to free up a bed in
    # In a more complex model, this would be the actual ward name
    heapq.heappush(event_list, (departure_time, 'DEPARTURE', patient_type))


# --- 4. Event Handlers ---

def handle_arrival(time, patient_type):
    """Logic for when a new patient arrives."""
    total_arrivals[patient_type] += 1
    
    # --- Primary Admission Attempt ---
    primary_ward = patient_type
    if occupied_beds[primary_ward] < capacities[primary_ward]:
        # Success! Bed available in the correct ward.
        occupied_beds[primary_ward] += 1
        successful_admissions[primary_ward] += 1
        schedule_departure(patient_type, time)
    else:
        # --- Relocation Attempt ---
        relocated_from[primary_ward] += 1
        
        # Choose an alternative ward based on relocation probabilities
        wards = list(reloc_probs[patient_type].keys())
        probs = list(reloc_probs[patient_type].values())
        alt_ward = np.random.choice(wards, p=probs)
        
        if occupied_beds[alt_ward] < capacities[alt_ward]:
            # Success! Bed available in the alternative ward.
            occupied_beds[alt_ward] += 1
            # Note: We still schedule the departure based on the patient_type's stay duration
            schedule_departure(patient_type, time)
        else:
            # Failure. Patient is lost.
            lost_patients[patient_type] += 1
            
    # Always schedule the next arrival for this patient type
    schedule_arrival(patient_type, time)


def handle_departure(time, patient_type):
    """Logic for when a patient leaves."""
    # This is simplified: we assume the patient frees a bed in their primary ward.
    # A more complex model would track which ward each patient is *actually* in.
    ward = patient_type 
    if occupied_beds[ward] > 0:
        occupied_beds[ward] -= 1


# --- 5. Main Simulation Loop ---

# Kickstart the simulation by scheduling the first arrival for each patient type
current_time = 0.0
for p_type in ward_names.values():
    schedule_arrival(p_type, current_time)

# Simulate for a total of 365 days
simulation_duration = 365
burn_in_period = 50 # Ignore stats from the first 50 days to let the system stabilize

print("Starting simulation...")
while current_time < simulation_duration:
    
    # Get the very next event from the list
    time, event_type, patient_type = heapq.heappop(event_list)
    current_time = time
    
    # Don't collect stats during the burn-in period
    if current_time < burn_in_period:
        if event_type == 'ARRIVAL':
            handle_arrival(time, patient_type)
        elif event_type == 'DEPARTURE':
            handle_departure(time, patient_type)
        continue # Skip to next event

    # Process the event
    if event_type == 'ARRIVAL':
        handle_arrival(time, patient_type)
    elif event_type == 'DEPARTURE':
        handle_departure(time, patient_type)

print("...Simulation finished.\n")


# --- 6. Report Results ---

print("--- Hospital Performance Results (Baseline) ---")
for ward in ward_names.values():
    arrivals = total_arrivals[ward]
    admissions = successful_admissions[ward]
    relocated = relocated_from[ward]
    lost = lost_patients[ward]
    
    # Avoid division by zero if there were no arrivals
    prob_full = (relocated / arrivals) * 100 if arrivals > 0 else 0
    prob_lost = (lost / arrivals) * 100 if arrivals > 0 else 0
    
    print(f"\n--- Ward {ward} ---")
    print(f"  Bed Capacity: {capacities[ward]}")
    print(f"  Total Arrivals (Type {ward}): {arrivals}")
    print(f"  Admissions to Primary Ward: {admissions}")
    print(f"  Relocated FROM this Ward: {relocated}")
    print(f"  Patients Lost after Relocation: {lost}")
    print(f"  Probability Ward is Full on Arrival: {prob_full:.2f}%")

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}

In [11]:
import numpy as np
import heapq

# ---
# Hospital Bed Allocation Simulation - Condensed & Structured Model
# ---

# ==============================================================================
# PART 1: Define the Hospital and its Rules (The Model)
# ==============================================================================

class HospitalModel:
    def __init__(self, capacities, arrival_rates, mean_stays, reloc_probs):
        """Initializes the hospital with its fixed parameters and current state."""
        self.capacities = capacities
        self.arrival_rates = arrival_rates
        self.mean_stays = mean_stays
        self.reloc_probs = reloc_probs
        self.wards = list(capacities.keys())

        # State variables that change during simulation
        self.occupied_beds = {ward: 0 for ward in self.wards}
        
        # Statistics trackers
        self.stats = {
            'total_arrivals': {ward: 0 for ward in self.wards},
            'primary_admissions': {ward: 0 for ward in self.wards},
            'relocated_from': {ward: 0 for ward in self.wards},
            'relocated_to': {ward: 0 for ward in self.wards},
            'lost_patients': {ward: 0 for ward in self.wards}
        }

    def handle_arrival(self, patient_type):
        """
        Handles a patient arrival. This contains the logic for direct admission,
        relocation (transfer), and lost patients.
        Returns the ward of admission, or None if the patient is lost.
        """
        self.stats['total_arrivals'][patient_type] += 1
        
        # --- Primary Admission Attempt ---
        primary_ward = patient_type
        if self.occupied_beds[primary_ward] < self.capacities[primary_ward]:
            # Success: Admit to the primary ward
            self.occupied_beds[primary_ward] += 1
            self.stats['primary_admissions'][primary_ward] += 1
            return primary_ward
        else:
            # Failure: Ward is full, attempt relocation (transfer)
            self.stats['relocated_from'][primary_ward] += 1
            return self._try_relocate(patient_type)

    def _try_relocate(self, patient_type):
        """Handles the logic for transferring a patient to an alternative ward."""
        # Choose an alternative ward based on probabilities
        wards = list(self.reloc_probs[patient_type].keys())
        probs = list(self.reloc_probs[patient_type].values())
        alt_ward = np.random.choice(wards, p=probs)
        
        if self.occupied_beds[alt_ward] < self.capacities[alt_ward]:
            # Success: Admit to the alternative ward
            self.occupied_beds[alt_ward] += 1
            self.stats['relocated_to'][alt_ward] += 1
            return alt_ward
        else:
            # Failure: Alternative ward is also full, patient is lost
            self.stats['lost_patients'][patient_type] += 1
            return None
            
    def handle_departure(self, ward_of_admission):
        """Handles a patient departure, freeing up a bed."""
        if self.occupied_beds[ward_of_admission] > 0:
            self.occupied_beds[ward_of_admission] -= 1


# ==============================================================================
# PART 2: Define the Simulation Engine
# ==============================================================================

class Simulation:
    def __init__(self, hospital_model, duration):
        """Initializes the simulation engine."""
        self.hospital = hospital_model
        self.duration = duration
        self.current_time = 0.0
        self.event_list = [] # A priority queue of future events

    def schedule_event(self, delay, event_type, details):
        """Adds a new event to the queue."""
        heapq.heappush(self.event_list, (self.current_time + delay, event_type, details))

    def run(self, burn_in=50):
        """Runs the entire discrete-event simulation."""
        
        # Kickstart by scheduling the first arrival for each patient type
        for p_type in self.hospital.wards:
            delay = np.random.exponential(1.0 / self.hospital.arrival_rates[p_type])
            self.schedule_event(delay, 'ARRIVAL', {'patient_type': p_type})

        # --- Main Simulation Loop ---
        while self.current_time < self.duration:
            # Get the next event
            time, event_type, details = heapq.heappop(self.event_list)
            self.current_time = time
            
            collect_stats = self.current_time > burn_in

            if event_type == 'ARRIVAL':
                patient_type = details['patient_type']
                
                # An arrival always triggers the *next* arrival for that type
                next_arrival_delay = np.random.exponential(1.0 / self.hospital.arrival_rates[patient_type])
                self.schedule_event(next_arrival_delay, 'ARRIVAL', {'patient_type': patient_type})

                # Handle the current arrival
                ward_admitted = self.hospital.handle_arrival(patient_type)
                
                # If admitted, schedule their departure
                if ward_admitted:
                    stay_duration = np.random.exponential(self.hospital.mean_stays[patient_type])
                    self.schedule_event(stay_duration, 'DEPARTURE', {'ward': ward_admitted})
            
            elif event_type == 'DEPARTURE':
                self.hospital.handle_departure(details['ward'])

        print("...Simulation finished.\n")
        
    def report_results(self):
        """Prints a summary of the simulation statistics."""
        print("--- Hospital Performance Results (Baseline) ---")
        stats = self.hospital.stats
        for ward in self.hospital.wards:
            arrivals = stats['total_arrivals'][ward]
            admissions = stats['primary_admissions'][ward]
            relocated = stats['relocated_from'][ward]
            lost = stats['lost_patients'][ward]
            
            prob_full = (relocated / arrivals * 100) if arrivals > 0 else 0
            
            print(f"\n--- Ward {ward} (Capacity: {self.hospital.capacities[ward]}) ---")
            print(f"  Total Arrivals (Type {ward}): {stats['total_arrivals'][ward]}")
            print(f"  Primary Admissions: {stats['primary_admissions'][ward]}")
            print(f"  Relocated FROM this Ward: {stats['relocated_from'][ward]}")
            print(f"  Patients Lost (Type {ward}): {stats['lost_patients'][ward]}")
            print(f"  Probability Ward is Full on Arrival: {prob_full:.2f}%")

# ==============================================================================
# PART 3: Set Up and Run the Simulation
# ==============================================================================

# Define all parameters
capacities = {'A': 55, 'B': 40, 'C': 30, 'D': 20, 'E': 20}
arrival_rates = {'A': 14.5, 'B': 11.0, 'C': 8.0, 'D': 6.5, 'E': 5.0}
mean_stays = {'A': 2.9, 'B': 4.0, 'C': 4.5, 'D': 1.4, 'E': 3.9}
reloc_probs = {
    'A': {'A': 0.0, 'B': 0.05, 'C': 0.10, 'D': 0.05, 'E': 0.80},
    'B': {'A': 0.2, 'B': 0.0, 'C': 0.50, 'D': 0.15, 'E': 0.15},
    'C': {'A': 0.3, 'B': 0.2, 'C': 0.0, 'D': 0.20, 'E': 0.30},
    'D': {'A': 0.35,'B': 0.3, 'C': 0.05, 'D': 0.0, 'E': 0.30},
    'E': {'A': 0.2, 'B': 0.1, 'C': 0.60, 'D': 0.1, 'E': 0.0}
}

# 1. Create an instance of the hospital model
hospital = HospitalModel(capacities, arrival_rates, mean_stays, reloc_probs)

# 2. Create an instance of the simulation engine
sim = Simulation(hospital, duration=365)

# 3. Run the simulation
sim.run()

# 4. Show the results
sim.report_results()

...Simulation finished.

--- Hospital Performance Results (Baseline) ---

--- Ward A (Capacity: 55) ---
  Total Arrivals (Type A): 5279
  Primary Admissions: 5027
  Relocated FROM this Ward: 252
  Patients Lost (Type A): 115
  Probability Ward is Full on Arrival: 4.77%

--- Ward B (Capacity: 40) ---
  Total Arrivals (Type B): 3942
  Primary Admissions: 3090
  Relocated FROM this Ward: 852
  Patients Lost (Type B): 278
  Probability Ward is Full on Arrival: 21.61%

--- Ward C (Capacity: 30) ---
  Total Arrivals (Type C): 2967
  Primary Admissions: 1866
  Relocated FROM this Ward: 1101
  Patients Lost (Type C): 231
  Probability Ward is Full on Arrival: 37.11%

--- Ward D (Capacity: 20) ---
  Total Arrivals (Type D): 2294
  Primary Admissions: 2248
  Relocated FROM this Ward: 46
  Patients Lost (Type D): 5
  Probability Ward is Full on Arrival: 2.01%

--- Ward E (Capacity: 20) ---
  Total Arrivals (Type E): 1855
  Primary Admissions: 1289
  Relocated FROM this Ward: 566
  Patients Lost (

In [17]:

import numpy as np

# ---
# Hospital Bed Allocation Simulation - Simplified for Clarity
# ---

# ==============================================================================
# PART 1: SETUP - Defining the Hospital's Rules and Initial State
# ==============================================================================
print("--- Part 1: Setting up the hospital model ---")

# --- Define the hospital's wards ---
wards = ['A', 'B', 'C', 'D', 'E']

# --- Define the fixed parameters of the hospital ---
# How many beds each ward has
capacities = {
    'A': 55, 'B': 40, 'C': 30, 'D': 20, 'E': 20
}
# How often new patients of each type arrive (in patients per day)
arrival_rates = {
    'A': 14.5, 'B': 11.0, 'C': 8.0, 'D': 6.5, 'E': 5.0
}
# How long patients of each type stay on average (in days)
mean_stay_duration = {
    'A': 2.9, 'B': 4.0, 'C': 4.5, 'D': 1.4, 'E': 3.9
}
# The probabilities for where to send a patient if their main ward is full
transfer_probabilities = {
    'A': {'A': 0.0, 'B': 0.05, 'C': 0.10, 'D': 0.05, 'E': 0.80},
    'B': {'A': 0.2, 'B': 0.0, 'C': 0.50, 'D': 0.15, 'E': 0.15},
    'C': {'A': 0.3, 'B': 0.2, 'C': 0.0, 'D': 0.20, 'E': 0.30},
    'D': {'A': 0.35,'B': 0.3, 'C': 0.05, 'D': 0.0, 'E': 0.30},
    'E': {'A': 0.2, 'B': 0.1, 'C': 0.60, 'D': 0.1, 'E': 0.0}
}

# --- Initialize variables to track the hospital's current state ---
# We use dictionaries to store values for each ward. Let's create them with a loop.
occupied_beds = {}
for ward_name in wards:
    occupied_beds[ward_name] = 0

# --- Initialize trackers to store statistics ---
# We will count every important outcome to analyze the results later.
total_arrivals = {}
primary_admissions = {}
patients_relocated = {}
patients_lost = {}
for ward_name in wards:
    total_arrivals[ward_name] = 0
    primary_admissions[ward_name] = 0
    patients_relocated[ward_name] = 0
    patients_lost[ward_name] = 0


# --- The Event List ---
# This is a simple list that will hold all future events.
# Each event will be a small dictionary: {'time': 1.23, 'type': 'ARRIVAL', 'details': 'A'}
event_list = []

print("...Setup complete.\n")


# ==============================================================================
# PART 2: CORE LOGIC - Defining What Happens for Each Event
# ==============================================================================

# This section contains the "rules" for arrivals, departures, and transfers.

def handle_arrival(patient_type, arrival_time):
    """
    This function contains the logic for when a patient arrives.
    It checks the primary ward first, then calls the transfer logic if needed.
    """
    global occupied_beds, total_arrivals, primary_admissions, patients_relocated
    
    # --- Simulate Patient Arrival ---
    total_arrivals[patient_type] += 1
    primary_ward = patient_type
    
    # Check if a bed is free in the patient's main ward
    if occupied_beds[primary_ward] < capacities[primary_ward]:
        # If yes, admit the patient
        occupied_beds[primary_ward] += 1
        primary_admissions[patient_type] += 1
        # Now that they are admitted, schedule when they will leave
        handle_departure(patient_type, primary_ward, arrival_time)
    else:
        # If the main ward is full, the patient needs to be relocated
        patients_relocated[patient_type] += 1
        handle_transfer(patient_type, arrival_time)

def handle_transfer(patient_type, arrival_time):
    """
    This function handles the logic for transferring (relocating) a patient.
    """
    global occupied_beds, patients_lost
    
    # Get the possible wards and probabilities for the transfer
    possible_wards = list(transfer_probabilities[patient_type].keys())
    probabilities = list(transfer_probabilities[patient_type].values())
    
    # Choose a random alternative ward
    chosen_ward = np.random.choice(possible_wards, p=probabilities)
    
    # Check if a bed is free in the chosen alternative ward
    if occupied_beds[chosen_ward] < capacities[chosen_ward]:
        # If yes, admit the patient to the alternative ward
        occupied_beds[chosen_ward] += 1
        # Schedule their departure from this alternative ward
        handle_departure(patient_type, chosen_ward, arrival_time)
    else:
        # If the alternative ward is also full, the patient is "lost"
        patients_lost[patient_type] += 1

def handle_departure(patient_type, admission_ward, admission_time):
    """
    This function handles scheduling a patient's departure.
    It calculates how long the patient will stay and adds a 'DEPARTURE' event
    to the event list for that future time.
    """
    global event_list
    
    # Calculate how long the patient's stay will be
    stay_duration = np.random.exponential(scale=mean_stay_duration[patient_type])
    
    # Calculate the exact time of departure
    departure_time = admission_time + stay_duration
    
    # Create the departure event and add it to our list
    departure_event = {
        'time': departure_time,
        'type': 'DEPARTURE',
        'details': {'ward_to_free': admission_ward} # We need to know which bed to free up
    }
    event_list.append(departure_event)

# ==============================================================================
# PART 3: THE SIMULATION ENGINE - Running the Model Over Time
# ==============================================================================
print("--- Part 3: Running the simulation ---")

# --- Kickstart the simulation ---
# Schedule the very first arrival for each type of patient.
current_time = 0.0
for patient_type in wards:
    # Calculate when the first patient of this type will arrive
    arrival_delay = np.random.exponential(scale=1.0 / arrival_rates[patient_type])
    first_arrival_event = {
        'time': current_time + arrival_delay,
        'type': 'ARRIVAL',
        'details': {'patient_type': patient_type}
    }
    event_list.append(first_arrival_event)

# --- Main Simulation Loop ---
simulation_duration = 365 # We will simulate for one year

while current_time < simulation_duration:
    
    # To get the next event, we sort the list by time and take the first one
    # This is less efficient than a priority queue, but much easier to understand
    event_list.sort(key=lambda event: event['time'])
    
    # Get the next event to process
    next_event = event_list.pop(0)
    
    # Advance the simulation clock to the time of this event
    current_time = next_event['time']
    
    # Stop if we have passed the total simulation duration
    if current_time > simulation_duration:
        break
        
    # --- Process the event based on its type ---
    event_type = next_event['type']
    event_details = next_event['details']
    
    if event_type == 'ARRIVAL':
        # When a patient arrives...
        patient_type = event_details['patient_type']
        
        # ...handle their admission or transfer process.
        handle_arrival(patient_type, current_time)
        
        # ...and immediately schedule the *next* arrival for that same patient type.
        # This ensures a continuous stream of new patients.
        next_arrival_delay = np.random.exponential(scale=1.0 / arrival_rates[patient_type])
        new_arrival_event = {
            'time': current_time + next_arrival_delay,
            'type': 'ARRIVAL',
            'details': {'patient_type': patient_type}
        }
        event_list.append(new_arrival_event)
        
    elif event_type == 'DEPARTURE':
        # When a patient's stay is over, free up their bed.
        ward_to_free = event_details['ward_to_free']
        if occupied_beds[ward_to_free] > 0:
            occupied_beds[ward_to_free] -= 1

print("...Simulation finished.\n")

# ==============================================================================
# PART 4: REPORTING - Analyzing the Results
# ==============================================================================
print("--- Part 4: Analyzing and Reporting Results ---")

for ward in wards:
    # Get the final counts from our trackers
    arrivals = total_arrivals[ward]
    admissions = primary_admissions[ward]
    relocated = patients_relocated[ward]
    lost = patients_lost[ward]
    
    # Calculate the probability that a patient arriving for this ward found it full
    # We check if arrivals is > 0 to avoid dividing by zero
    if arrivals > 0:
        prob_full = (relocated / arrivals) * 100
    else:
        prob_full = 0
    
    print("-----------------------------------------")
    print(f"Results for Ward {ward}")
    print("-----------------------------------------")
    print(f"  Bed Capacity: {capacities[ward]}")
    print(f"  Total Patient Arrivals for this Ward: {arrivals}")
    print(f"  Patients Admitted to this Ward Directly: {admissions}")
    print(f"  Patients Turned Away (Ward Full): {relocated}")
    print(f"  Patients Lost After Being Turned Away: {lost}")
    print(f"  Probability Ward was Full on Arrival: {prob_full:.2f}%")



--- Part 1: Setting up the hospital model ---
...Setup complete.

--- Part 3: Running the simulation ---
...Simulation finished.

--- Part 4: Analyzing and Reporting Results ---
-----------------------------------------
Results for Ward A
-----------------------------------------
  Bed Capacity: 55
  Total Patient Arrivals for this Ward: 5357
  Patients Admitted to this Ward Directly: 5121
  Patients Turned Away (Ward Full): 236
  Patients Lost After Being Turned Away: 111
  Probability Ward was Full on Arrival: 4.41%
-----------------------------------------
Results for Ward B
-----------------------------------------
  Bed Capacity: 40
  Total Patient Arrivals for this Ward: 4020
  Patients Admitted to this Ward Directly: 3190
  Patients Turned Away (Ward Full): 830
  Patients Lost After Being Turned Away: 261
  Probability Ward was Full on Arrival: 20.65%
-----------------------------------------
Results for Ward C
-----------------------------------------
  Bed Capacity: 30
  Total